In [16]:
import requests
import re
import os
import openai

#return the api found in the url
def get_api(url):
    response = requests.get(url)
    text = response.text
    start = text.find("<em>")
    end = text.find("</em>")
    text = text[start:end]
    text = re.sub("<em>|<br/>", "", text)
    return text

In [18]:
# API endpoint for text-davinci-003
endpoint = "https://api.openai.com/v1/engines/text-davinci-003/jobs"

# Your API key
api_key = get_api("https://alignmentjam.com/oversight")


openai.api_key = os.getenv("OPENAI_API_KEY")

response = openai.Completion.create(
  model="text-davinci-003",
  prompt="Decide whether a Tweet's sentiment is positive, neutral, or negative.\n\nTweet: \"I loved the new Batman movie!\"\nSentiment:",
  temperature=0,
  max_tokens=60,
  top_p=1,
  frequency_penalty=0.5,
  presence_penalty=0
)


Failed to generate text. Response status code: 403
